# Features

data_stocks.tickers['ADS.DE'].calendar.index.rename('date', inplace=True)

df.fillna(method='ffill', limit=max_forward, inplace=True)

weekday diff_days 
stock = stock.reset_index(level=0)
stock['Daily Change'] = stock['Adj. Close'] - stock['Adj. Open']

n_shares = math.floor(money / start_price)
end_price = (price_high + open_price + close_price) / 3
profit = n_shares * (end_price - start_price)

primary key -> remove duplicates (same dates)

* data_stocks
    * tickers:
        * name e.g. ADS.DE
            * calendar
                * Earnings Date -> date
                * earnings_average
                * earnings_low
                * earnings_high
                * revenue_average
                * revenue_low
                * revenue_high
            * recommendations
                * Date -> date
                * firm: Berenberg, Exane BNP Paribas
                * to_grade: Buy, Neutral
                * from_grade: Hold, ''
                * action: up, init
            * info
                * 52WeekChange
                * SandP52WeekChange
                * averageDailyVolume10Day
                * averageVolume
                * averageVolume10days
                * beta
                * ask
                * askSize
                * bid
                * bidSize
                * bookValue
                * dayHigh
                * dayLow
                * dividendRate
                * dividendYield
                * earningsQuarterlyGrowth
                * enterpriseToEbitda
                * enterpriseToRevenue
                * enterpriseValue
                * exDividendDate
                * fiftyDayAverage
                * fiftyTwoWeekHigh
                * fiftyTwoWeekLow
                * fiveYearAvgDividendYield
                * floatShares
                * forwardEps
                * forwardPE
                * fullTimeEmployees
                * heldPercentInsiders
                * heldPercentInstitutions
                * marketCap
                * netIncomeToCommon
                * open
                * payoutRatio
                * pegRatio
                * previousClose
                * priceToBook
                * profitMargins
                * sector: Consumer Cyclical
                * sharedOutstanding
                * symbol
                * regularMarketDayHigh
                * regularMarketDayLow
                * regularMarketOpen
                * regularMarketPreviousClose
                * regularMarketVolume
                * trailingAnnualDividendRate
                * trailingAnnualDividendYield
                * trailingEps
                * trailingPE
                * volume               
            * sustainability
                * Date -> date
                * palmoil: t/f
                * controversialweapons: t/f
                * gambling: t/f
                * socialscore: float
                * nuclear: t/f
                * furleather: t/f
                * alcoholic: t/f
                * gmo: t/f
                * catholic: t/f
                * socialpercentile: float
                * peercount: int
                * relatedcontroversy: text
                * governancescore: float
                * environmentpercentile: int
                * animaltesting: t/f
                * tobacco: t/f
                * totalesg: float
                * highestcontroversy: int
                * esgperformance: OUT_PERF
                * coal: t/f
                * pesticides: t/f
                * adult: t/f
                * percentile: float
                * peergroup: Textiles & Apparel
                * smallarms: t/f
                * environmentscore: float    
                * governancepercentile: float
                * militarycontract: t/f
            * earnings
                * Quarter -> date
                * revenue
                * earnings
            * financials (quarter)
                * "" -> date
                * research_development: None
                * effect_of_accounting_charges: None
                * income_before_tax: float
                * minority_interest: float (neg)
                * net_income: float
                * selling_general_administrative: float
                * gross_profit: float
                * ebit: float
                * operating_income: float
                * other_operating_expenses: float (neg)
                * interest_expense: float (neg)
                * extraordinary_items: None
                * non_recurring: None
                * other_items: None
                * income_tax_expense: float
                * total_revenue: float
                * total_operating_expenses: float
                * cost_of_revenue: float
                * total_other_income_expense_net: float (neg)
                * discontinued_operations: float
                * net_income_from_continuing_ops: float
                * net_income_applicable_to_common_shares: float
            * balancesheet (quarter)
                * "" -> date
                * intangible_assets: float
                * total_liab: float
                * total_stockholder_equity: float
                * minority_interest
                * other_current_liab
                * total_assets
                * common_stock
                * other_current_assets
                * retained_earnings
                * other_liab
                * good_will
                * treasury_stock
                * other_assets
                * cash
                * total_current_liabilities
                * deferred_long_term_asset_charges
                * short_long_term_debt
                * other_stockholder_equity
                * property_plant_equipment
                * total_current_assets
                * long_term_investments
                * net_tangible_assets
                * short_term_investments
                * net_receivables
                * long_term_debt
                * inventory
                * accounts_payable
            * cashflow (quarter)
                * investments
                * change_to_liabilities
                * total_cashflows_from_investing_activities
                * net_borrowings
                * total_cash_from_financing_activities
                * issuance_of_stock
                * net_income
                * change_in_cash
                * repurchase_of_stock
                * effect_of_exchange_rate
                * total_cash_from_operating_activities
                * depreciation
                * other_cashflows_from_investing_activities
                * change_to_inventory
                * change_to_account_receivables
                * other_cashflows_from_financing_activities
                * change_to_netincome
                * capital_expenditures
                * dividends_paid
            * dividends: EMPTY
            * splits: EMPTY
            * history
                * Date -> date
                * open
                * high
                * low
                * close
                * dividends
                * stock_splits
            * calls: EMPTY
            * puts: EMPTY
    * downloads:
        * name e.g. ADS.DE
            * close
                * Date -> date
                * close
            * high
                * Date
                * high
            * low
                * Date
                * low
            * open
                * Date
                * open
            * volume
                * Date
                * volume
    
* benchmark .. * 90
    * tickers: 
        * same as stocks
    * downloads:
        * name e.g. ADS.DE
            * close
                * Date -> date
                * close
            * high
                * Date
                * high
            * low
                * Date
                * low
            * open
                * Date
                * open
            * volume
                * Date
                * volume

In [1]:
import numpy as np
import pandas as pd
from importlib import reload

import shared
import config
import provider_yfinance as provider
import munch

reload(shared)
reload(config)
reload(provider)

cfg = config.get_config('^GDAXI')

# use cached data
cfg.prepare.download_end_dt = '2019-12-19'

config> read from config file: 'Munch({'base': Munch({'cache_dir': '/mnt/c/notebooks/sandbox/cache', 'cache_enabled': False, 'config_file_path': '/mnt/c/notebooks/sandbox/config.json', 'current_dir': '/mnt/c/notebooks/sandbox', 'model_dir': '/mnt/c/notebooks/sandbox/model'}), 'datasets': Munch({'raw': Munch({'benchmarks': ['000001.SS', 'AUDUSD=X', 'BTCUSD=X', 'CNY=X', 'ETHUSD=X', 'EURCAD=X', 'EURCHF=X', 'EURGBP=X', 'EURHUF=X', 'EURJPY=X', 'EURJPY=X', 'EURSEK=X', 'EURUSD=X', 'GBPJPY=X', 'GBPUSD=X', 'GC=F', 'HKD=X', 'IDR=X', 'IMOEX.ME', 'INR=X', 'JPY=X', 'KW=F', 'LC=F', 'LH=F', 'MXN=X', 'MYR=X', 'NZDUSD=X', 'PA=F', 'PHP=X', 'PL=F', 'RUB=X', 'S=F', 'SGD=X', 'SI=F', 'SM=F', 'THB=X', 'ZAR=X', '^AORD', '^AXJO', '^BFX', '^BSESN', '^BUK100P', '^BVSP', '^DJI', '^FCHI', '^FTSE', '^GDAXI', '^GSPC', '^GSPTSE', '^HSI', '^IPSA', '^IXIC', '^JKSE', '^JN0U.JO', '^KLSE', '^KS11', '^MERV', '^MXX', '^N100', '^N225', '^NYA', '^NZ50', '^RUT', '^STI', '^STOXX50E', '^TA125.TA', '^TWII', '^VIX', '^XAX'], 'stoc

In [2]:
cfg_stocks, data_stocks = provider.load_stocks(cfg)
cfg_benchmarks, data_benchmarks = provider.load_benchmarks(cfg)

In [3]:
prep_stocks = {}
for k in data_stocks.tickers.keys():
    df = data_stocks.tickers[k].history.copy()
    df = df.loc[df.index >= cfg.prepare.data_start_dt]
    df = shared.generate_dt(cfg, df)
    df = shared.generate_diff(cfg, df)
    df = shared.generate_rolling_windows(cfg, df)
    df = shared.generate_loglag(cfg, df)
    df = df.loc[df.index >= cfg.train.start_dt]
    df.break_days.astype[int]
    prep_stocks[k] = df
shared.save_pickle(f'{cfg.prepare.cache_dir}/stocks_prep.pkl', prep_stocks)
prep_stocks = munch.munchify(prep_stocks)

In [7]:
prep_benchmarks = {}
for k in data_benchmarks.tickers.keys():
    df = data_benchmarks.tickers[k].history.copy()
    df = df.loc[df.index >= cfg.prepare.data_start_dt]
    df = shared.generate_diff(cfg, df)
    df = shared.generate_rolling_windows(cfg, df)
    df = shared.generate_loglag(cfg, df)
    df = df.loc[df.index >= cfg.train.start_dt]
    prep_benchmarks[k] = df
shared.save_pickle(f'{cfg.prepare.cache_dir}/benchmarks_prep.pkl', prep_benchmarks)
prep_benchmarks = munch.munchify(prep_benchmarks)

In [6]:
prep_data['1COV.DE']

,open,high,low,close,volume,dividends,stock_splits,break_days,weekday,diff_prev,...,rolling_mean_3,rolling_mean_5,rolling_mean_7,rolling_mean_9,rolling_mean_21,rolling_mean_35,rolling_mean_50,lag_log_1,lag_log_2,lag_log_3
date,,,,,,,,,,,,,,,,,,,,,
2019-05-23,42.61,42.61,40.83,41.08,2613535,0.0,0,0.0,3,1.66,...,42.660000,43.038,43.485714,43.413333,45.910952,48.108286,47.4668,-0.052631,-0.059536,-0.046144
2019-05-24,41.35,41.52,40.92,40.95,1219570,0.0,0,0.0,4,-0.11,...,41.776667,42.390,43.012857,43.183333,45.421905,47.880000,47.3662,-0.003170,-0.055801,-0.062705
2019-05-27,41.25,41.59,40.92,41.46,562050,0.0,0,2.0,0,0.13,...,41.163333,42.078,42.514286,42.978889,44.991429,47.615429,47.2672,0.012377,0.009208,-0.043424
2019-05-28,41.35,41.80,41.04,41.12,2647188,0.0,0,0.0,1,1.26,...,41.176667,41.582,42.075714,42.630000,44.515238,47.350000,47.1584,-0.008234,0.004143,0.000973
2019-05-29,40.63,40.68,39.69,40.09,1758087,0.0,0,0.0,2,0.23,...,40.890000,40.940,41.657143,42.090000,44.020476,47.046286,47.0222,-0.025368,-0.033602,-0.021225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-13,43.92,44.54,42.82,42.82,1618272,0.0,0,0.0,4,0.97,...,42.446667,42.212,42.080000,41.968889,42.752381,43.781714,44.1968,0.001168,0.025306,0.022675
2019-12-16,43.00,43.41,42.83,42.95,892510,0.0,0,2.0,0,1.04,...,42.846667,42.430,42.262857,42.153333,42.660952,43.713429,44.1906,0.003031,0.004200,0.028337
2019-12-17,43.06,43.06,41.79,41.96,1395932,0.0,0,0.0,1,1.21,...,42.576667,42.450,42.281429,42.163333,42.561429,43.631143,44.1854,-0.023320,-0.020288,-0.019120


In [9]:
prep_benchmarks['000001.SS']

,open,high,low,close,volume,dividends,stock_splits,diff_prev,diff_oc,diff_hl,rolling_mean_3,rolling_mean_5,rolling_mean_7,rolling_mean_9,rolling_mean_21,rolling_mean_35,rolling_mean_50,lag_log_1,lag_log_2,lag_log_3
date,,,,,,,,,,,,,,,,,,,,
2019-05-23,2880.84,2885.14,2846.96,2852.51,197300,0,0,27.84,-28.33,38.18,2883.396667,2880.618,2899.640000,2898.311111,2997.525714,3075.522286,3067.8548,-0.013649,-0.018568,-0.006322
2019-05-24,2847.84,2871.86,2846.02,2853.00,167200,0,0,-44.54,5.16,25.84,2865.740000,2874.758,2887.400000,2892.676667,2978.611429,3071.466857,3062.7864,0.000172,-0.013477,-0.018396
2019-05-27,2851.28,2898.13,2833.04,2892.38,196700,0,0,-58.63,41.10,65.09,2865.963333,2879.114,2878.352857,2893.651111,2960.591429,3065.798857,3061.2368,0.013709,0.013880,0.000232
2019-05-28,2890.27,2924.04,2887.08,2909.91,223300,0,0,-24.43,19.64,36.96,2885.096667,2879.902,2882.297143,2890.454444,2946.061429,3058.357429,3058.8952,0.006042,0.019751,0.019923
2019-05-29,2894.83,2934.98,2890.67,2914.70,199000,0,0,-10.97,19.87,44.31,2905.663333,2884.500,2888.597143,2885.897778,2932.542857,3050.868286,3055.9830,0.001645,0.007687,0.021396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-12,2926.34,2926.34,2913.48,2915.70,151800,0,0,-41.34,-10.64,12.86,2919.146667,2916.786,2908.788571,2902.447778,2902.119048,2922.251714,2930.9124,-0.002986,-0.000555,0.000419
2019-12-13,2937.78,2969.98,2935.85,2967.68,212000,0,0,-46.61,29.90,34.13,2935.933333,2927.920,2921.582857,2912.655556,2904.871905,2922.616000,2931.6226,0.017671,0.014684,0.017115
2019-12-16,2970.97,2984.64,2958.71,2984.39,211900,0,0,-51.45,13.42,25.93,2955.923333,2941.902,2933.714286,2923.732222,2909.302857,2922.740000,2933.2066,0.005615,0.023285,0.020299


In [5]:
df = pd.merge(history, b_history, how='left', left_index=True, right_index=True)
df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)
df

NameError: name 'history' is not defined